## Levatamos Fermionic

In [1]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
import functools
import concurrent.futures
from numba import njit
import tensorflow as tf
import scipy
import sparse
import itertools
import linecache
from multiprocessing import Pool, cpu_count

# Generación de base
class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)

    def idx_to_repr(self, idx):
        return self.canonicals[idx]

    def opr_to_idx(self, opr):
        for i in range(self.size): # Evitar esto ordenando opr
            if self.base[i] == opr:
                return i

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None, pairs = False):
        basis = []
        num_ele = []
        for k in range(0,2**d):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    if pairs:
                        if np.all(b[::2] == b[1::2]):
                            oper = self.bin_to_op(b)
                            basis.append(oper)
                            num_ele.append(k)
                    else:
                        oper = self.bin_to_op(b)
                        basis.append(oper)
                        num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def __init__(self, d, num = None, pairs = False, basis = None, num_ele = None):
        self.d = d
        self.num = num
        self.m = num
        # Si nos da la base, la levantamos (asumimos GC). Si no, la creamos
        if basis is None:
            self.base, self.num_ele = self.create_basis(d, num, pairs)
        else:
            self.base, self.num_ele = basis, num_ele
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        self.pairs = pairs

    @staticmethod
    def cdc(i, j):
        return of.FermionOperator(((i,1),(j,0)))

    @staticmethod
    def cc(i, j):
        return of.FermionOperator(((i,0),(j,0)))

    # Del indice, cuenta el número de partículas
    def num_idx(self, idx):
        b = self.int_to_bin(idx, basis.d)
        return b.count('1')

    # Calculo de rho1 (via directa, lento, y solo definido en la base por ahora)
    def rho_1(self, op):
        # Necesitamos un índice, es?
        if type(op) != int:
            op = self.opr_to_idx(op)
        mat = np.zeros((self.d, self.d))
        for i in range(self.d):
            for j in range(self.d):
                cdc = self.cdc(j, i)
                mat[i,j] = self.idx_mean_val(op, cdc)
        return mat

# Calculo de generadores de rho1
def rho_1_gen(basis):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    indices = []
    values = []
    shape = (basis.d, basis.d, basis.size, basis.size)
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            # Generamos el operador
            op = basis.cdc(j, i)
            #print(op)
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# Calculo de rho1 (via generadores) de un vector en la base canonica
def rho_1(vect, rho_1_arrays):
    if len(vect.shape) == 1: # vectores
        return sparse.einsum('k,ijkl,l->ij', vect, rho_1_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad
        return sparse.einsum('ijkl,kl->ij', rho_1_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_1_arrays)

# Calculo de indices de rho2kkbar
def get_kkbar_indices(t_basis):
    indices = []
    for i, ind in enumerate(t_basis.num_ele):
        v = t_basis.int_to_bin(ind, t_basis.d)
        if np.all(v[::2] == v[1::2]):
            indices.append(i)
    return indices

# Calculo de generadores de rho2
def rho_2_gen(basis, t_basis, idx_list = []):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    d = basis.d
    indices = []
    values = []
    if len(idx_list) == basis.m:
        idx_list = idx_list
    elif len(idx_list) == basis.m**4:
        idx_list = np.unique(idx_list[:,0])
    else:
        idx_list = range(t_basis.size)
    shape = (len(idx_list), len(idx_list), basis.size, basis.size)
    for i, ii in tqdm(enumerate(idx_list), total=len(idx_list)):
        for j, jj in enumerate(idx_list):
            # Generamos el operador
            op = t_basis.base[jj]*of.utils.hermitian_conjugated(t_basis.base[ii])
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# rho_m_gen aux func
def process_chunk(args):
    chunk, m_basis, basis, d, it_set = args
    indices = []
    values = []
    for ii in chunk:
        for jj in it_set:
            # Generate the operator
            op = m_basis.base[jj] * of.utils.hermitian_conjugated(m_basis.base[ii])
            mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extract the information
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([ii, jj, r, c])
                values.append(v)
    return indices, values

# Parallelized rho_m_gen
def rho_m_gen(basis, m, num_workers=None):
    if num_workers is None:
        num_workers = cpu_count()  # Use all available CPUs by default
    
    indices = []
    values = []
    m_basis = fixed_basis(basis.d, num=m, pairs=basis.pairs)
    shape = (m_basis.size, m_basis.size, basis.size, basis.size)

    it_set = np.arange(m_basis.size)
    chunks = np.array_split(it_set, num_workers)  # Split `it_set` into chunks for each worker

    # Use multiprocessing Pool for parallel processing
    with Pool(processes=num_workers) as pool:
        # Pass arguments as tuples instead of using a lambda
        results = list(
            tqdm(
                pool.imap(
                    process_chunk, 
                    [(chunk, m_basis, basis, basis.d, it_set) for chunk in chunks]
                ),
                total=num_workers
            )
        )
    
    # Collect results from all processes
    for indices_chunk, values_chunk in results:
        indices.extend(indices_chunk)
        values.extend(values_chunk)

    # Construct the sparse array
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m(vect, rho_m_arrays):
    return sparse.einsum('k,ijkl,l->ij', vect, rho_m_arrays, vect)

# Calculo de rho2 (via generadores) de un estado en la base canonica
def rho_2(vect, rho_2_arrays):
    if len(vect.shape) == 1: # vectores SOLO RHO2 COMPLETA
        return sparse.einsum('k,ijkl,l->ij', vect, rho_2_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad SOLO RHO2 COMPLETA
        return sparse.einsum('ijkl,kl->ij', rho_2_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_2_arrays)

# Calculo de generadores de K (usado para quasiparticles) WIP SPARSE
def k_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d), total=d):
        for j in range(0, d):
            op = basis.cc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

def k_vect(vect, k_gen):
    return np.einsum('k,ijkl,l->ij', vect, k_gen, vect)

# Calculo la matrix rho de cuasipartículas  WIP SPARSE
def rho_qsp(vect, rho_1_arrays, k_arrays, rho1 = None):
    if type(rho1) == None:
        rho1 = rho_1(vect, rho_1_arrays)
    k = k_vect(vect, k_arrays)

    mat = np.block([[rho1, k], [-np.conjugate(k), np.eye(rho_1_arrays.shape[0])-np.conjugate(rho1)]])
    return mat

# Devuelve los indices que tienen a level ocupado
def level_proy(d, level):
    ids = []
    for k in range(0,2**d):
        b = fixed_basis.int_to_bin(k, d)
        if b[level] == '1':
            ids.append(k)
    arr = np.zeros(2**d)
    arr[np.array(ids)] = 1
    return arr, ids

def parity_levels(d):
    rng = range(2**d)
    binary_repr = np.vectorize(np.binary_repr)(rng)
    ones_c = np.char.count(binary_repr, '1')
    return np.array(rng)[ones_c % 2 == 1] # seleccionamos estados impares

# Devuelve el vector postmedido
def measure(basis, vect, level = 1):
    l_arr, l_ids = level_proy(basis.d, level)
    proy_v = vect * l_arr
    comp_arr = np.logical_not(l_arr).astype(int)
    comp_v = vect * comp_arr
    norm = lambda v: v / np.linalg.norm(v)
    return norm(proy_v), norm(comp_v)

def entropy(rho, m):
    S_fun = lambda rho: -1*np.trace(rho @ scipy.linalg.logm(rho)) / np.log(2)
    ent = S_fun(rho) / (np.log2(scipy.special.binom(basis.d, m)))
    return ent

# Levanta bases de QChem
def build_csv_basis(csvf, d):
    # Construimos la base
    ops = []
    with open(csvf, 'r') as basis:
        num_ele = []
        # Contar los niveles
        m_level = 0
        # Creamos operadores
        for l in basis.read().splitlines()[4:]:
            natop = [int(x) for x in l.split(' ')[1:]] # Operador en forma de lista
            #print(natop)
            op = of.FermionOperator(([(i, 1) for i in natop]))
            ops.append(op)
            # Contamos niveles
            m_level = max(m_level, *natop)
            # Determinamos el índice
            natop_to_int = lambda x: np.sum([2**(d-1-i) for i in x])
            num_ele.append(natop_to_int(natop))
 
        # Determinamos m y d
        num = len(natop)
        #assert d == m_level+1

    return fixed_basis(d, num = num, pairs = False, basis = ops, num_ele=num_ele)
        

2024-12-20 12:56:23.941053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734699384.003786    1289 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734699384.022867    1289 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 12:56:24.153348: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
basis = build_csv_basis('/home/debian/h2o_basis', 16)

rho_1_arrays = rho_m_gen(basis, 1)
rho_2_arrays = rho_m_gen(basis, 2)
# Indices
t_basis = fixed_basis(basis.d, num=2)
k_indices = get_kkbar_indices(t_basis)
print(k_indices)
# Arrays reducidos
#rho_2_kkbar_arrays = rho_2_gen(basis, t_basis, idx_list = k_indices)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [01:14<00:00,  4.68s/it]


[0, 5, 14, 27, 44, 65, 90, 119]


### Determinación de estado

In [4]:
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.cluster.vq import kmeans, vq

# Input
rho_1_obj  = np.sort(np.concatenate([np.repeat(1/2, 2*4), np.repeat(1, 2*3), np.repeat(0, 2)]))
rho_2_obj = np.sort(np.concatenate([np.repeat(1/12, 2*4+4), np.repeat(1/3, 2*2+2), np.repeat(1/2, 2*12+24), np.repeat(1, 2*3+9), np.repeat(3/4, 4), np.repeat(0,35)]))
vect = [ -1.040817778497415e-02,5.875375724324662e-11,-4.872095117042884e-02,6.873428921247475e-02,-1.221676948455208e-10,-2.838274900928131e-01,-5.878063978261826e-11,-8.645699663910012e-10,-5.775895570433762e-11,1.723160524415859e-10,-4.130668368087360e-04,-1.173087855953530e-10,-1.453056639505890e-04,-2.762016252654692e-11,-2.265626836499802e-11,-3.349465934536996e-06,-2.864495241964992e-12,-6.267691354963947e-07,-5.339978366042173e-13,8.104747583997980e-12,6.230579178507115e-13,3.086337355930937e-11,2.292959188928211e-03,-1.623821734917592e-11,3.528323278785784e-12,2.880945119699020e-01,1.200932893902544e-11,6.003848987292210e-11,1.703809642731980e-10,7.024650222678782e-10,1.780297002922397e-11,-1.215044751885651e-11,-8.802591465425141e-05,1.221054471875727e-10,1.236869342692051e-03,-1.095812765765958e-11,-3.147808351610409e-12,-1.620797257637043e-06,-8.333478763021257e-13,8.408255037214029e-06,4.949100623919174e-12,5.846465356141323e-07,-4.872095123122262e-02,1.863698523379154e-11,4.489548352478508e-03,5.719220024289879e-01,-7.325935764488445e-12,3.857553821498268e-02,1.584863299967308e-10,-8.270130853018783e-11,3.250336983023162e-12,8.011859892934100e-10,5.727962450765951e-05,4.646239209499062e-12,1.084015836844878e-05,9.445160896486132e-12,1.718759899952470e-11,-9.940417705918220e-07,-3.517578369792090e-11,-5.131396552130979e-08,-1.447788284202552e-12,-1.318437041398421e-12,-3.344505875990762e-13,6.873428914841577e-02,3.944264711681739e-11,5.719220018298049e-01,4.284422443311408e-03,-8.541947956013872e-11,-2.562174930607295e-02,6.741298638305758e-11,-1.485183361442374e-10,-8.306214037354152e-11,3.878837144087913e-11,-3.708999688758351e-04,1.098553067253537e-11,2.690496849475714e-03,-1.001678240733643e-11,-4.128191370862719e-11,-1.902693109981439e-06,2.973201919110332e-12,1.858192596490293e-05,5.337511916663943e-12,-1.347235271915940e-13,-7.947953562570418e-13,-1.398684089281688e-10,2.880945118129717e-01,4.936701642699172e-11,-1.203366754594425e-10,2.309214808193917e-03,-4.153538929856837e-11,-5.097968375149308e-11,1.320979042102519e-10,-4.465763917202127e-11,-7.715416033420891e-11,-2.361269457597528e-11,3.732270893980169e-05,-1.932917556558859e-11,-4.296016647493072e-04,-6.103562657998557e-11,-8.106491916782201e-14,7.124580458948895e-07,-5.391745840325733e-12,-3.252656274742233e-06,7.705230947359613e-13,-7.509190016417286e-07,-2.838274900815549e-01,1.361795229710217e-10,3.857553814359221e-02,-2.562174912588815e-02,-1.349168843204038e-10,-4.860386163550552e-03,-1.678635900907008e-11,-8.635566840102151e-11,-2.711893014341147e-11,8.570395066558972e-11,1.453627420713848e-03,2.444205956885194e-11,2.371941917405646e-04,2.009276972322758e-12,-1.511345484808334e-11,1.017368577569630e-05,7.184946196886101e-12,9.860328060251141e-07,2.567452881638662e-12,1.032351724868009e-11,-2.466393966698132e-12,-1.585674661535451e-11,-5.457166193858620e-11,-1.739551772237277e-11,-5.748327085059025e-11,-3.637001552998833e-11,-5.942288344452852e-11,1.071257324409827e-04,5.915118463973037e-11,-7.310205872880478e-04,2.240896895376143e-12,1.135970222615302e-11,7.498106556283097e-12,2.996392199114782e-11,4.314139288483464e-11,-2.787982729540680e-12,-5.878445673574833e-12,-1.676589056130999e-12,-9.376681213926539e-13,5.878634199600110e-13,7.426229122429301e-12,1.231298353383713e-13,2.949399690703477e-10,-3.917690713301482e-11,1.145144225687370e-10,9.384421996347579e-11,-4.883719259689913e-11,2.910600116630615e-11,8.975103079944680e-11,1.321991090091071e-04,-3.132958503562937e-11,-9.480533965877158e-04,-2.411313204451442e-11,-4.026699599855718e-11,-1.351128709309473e-11,6.009642850376845e-11,3.234438991702277e-07,4.541824089894673e-12,1.077572604517742e-12,-2.449242489829911e-12,-2.372208578401482e-12,-1.952236452060097e-09,-4.604178123159132e-13,1.177569767109985e-10,-1.902733587407053e-10,-1.254586491106212e-11,-1.439952530132711e-11,-1.100593008118344e-11,1.159101001879692e-10,-7.310207807367473e-04,1.007731340912288e-10,-4.065615358275551e-05,6.752887414168183e-12,-1.743006660097304e-11,-1.291220504861744e-12,3.961841356184016e-11,1.714835715387049e-11,-2.602268934662094e-12,-3.115462919429360e-12,2.902045052946052e-12,-1.366360510162913e-12,-6.433995352666110e-12,-3.487309586334203e-12,-2.813167764836577e-12,-1.379034101470578e-10,-4.814300245190187e-11,-7.788891180752160e-10,-4.166007838089737e-11,-7.254570421199601e-12,-4.763834394856907e-11,6.433616871757992e-12,-9.480534092964867e-04,-4.402930418738380e-11,-6.893939175172551e-05,-5.517221117995845e-11,-2.308453280832650e-11,1.437557867172997e-10,1.346315529218655e-11,-5.139343681390935e-06,-3.777610673719760e-13,-8.085362059414594e-12,-9.207654096585646e-13,-3.735654638369629e-12,-2.347916910826854e-09,-1.095394974662464e-12,-4.130668458720446e-04,-1.349081079938997e-11,5.727963161707341e-05,-3.708999735148822e-04,-1.606269977390437e-12,1.453627438030121e-03,1.115480324719588e-11,5.752687411208624e-11,-1.484341869147709e-11,4.749538621162773e-11,1.492647952480620e-04,-6.712706736797008e-11,-9.778164119893708e-04,-1.563989063445501e-11,-1.985777921167708e-11,3.649689214366206e-06,-3.761008943238818e-12,-2.361188027665020e-05,-4.602718816714851e-12,-2.629246254619853e-12,-1.643832607756848e-11,-1.142249188092814e-10,-8.802603583148170e-05,3.564050129045889e-12,-2.594162802453274e-11,3.732270720332251e-05,-1.860172782632815e-11,-1.326384570942282e-10,-3.533922533148709e-11,-1.151160659994413e-11,-5.454817472660802e-11,-2.238375891091281e-11,1.674296822403760e-04,-1.321748103833085e-11,-1.271534571081374e-03,5.472241384500912e-11,-3.052693261490014e-14,3.888251673178786e-06,5.793628752607294e-12,-3.152305206905967e-05,-8.110842301739709e-14,-1.192099148273008e-08,-1.453055462593280e-04,1.184109709033793e-10,1.084015297362576e-05,2.690496891118551e-03,2.658408207977311e-11,2.371941895595679e-04,-1.004464379148724e-10,1.411028031092301e-11,3.086029178734183e-11,-1.980213313522789e-11,-9.778165017831379e-04,8.636710083442739e-12,-6.615978238477744e-05,1.898770320901288e-11,3.694230910749729e-11,-2.419651843529180e-05,1.176662213576499e-11,-1.831170760561877e-06,1.542072257195798e-12,1.423099405202994e-12,1.599817214748861e-12,-1.382213663474287e-12,1.236869351070984e-03,-4.312411819232911e-12,-1.100572860455937e-11,-4.296016617354108e-04,1.381275152490570e-12,-4.517128957960822e-11,3.049871876562809e-11,-2.323939368869917e-11,6.187231585171055e-12,9.540585470819708e-12,-1.271534664205701e-03,3.330539756604071e-11,-9.456147117435777e-05,7.378258914940364e-12,-3.959550470631780e-12,-3.077214050263764e-05,9.967958127429818e-12,-2.090178340999786e-06,5.953970308336877e-12,1.485008502950520e-07,5.356211254317093e-11,-1.495190086377795e-11,-1.897203555542404e-11,4.296019654944173e-11,-2.867057860321690e-11,2.618459407621963e-11,-3.764820588801181e-12,3.234395160602702e-07,-3.864974309144221e-12,-5.139346903909730e-06,1.579844196217019e-11,-1.398952434566216e-11,3.251734020496331e-11,-4.861685553970752e-12,-2.724610477549601e-06,-2.238888007724852e-12,-3.415536587367198e-12,-5.057689161206319e-12,7.119405466816135e-12,-7.742745175471037e-08,2.184174763435573e-11,-3.349466932684585e-06,1.557628101134982e-12,-9.940207416660333e-07,-1.902682429699013e-06,-1.754496783340690e-12,1.017367401436453e-05,-4.897872272307307e-12,-1.485615040808395e-12,-2.862545817742472e-12,-1.549544500413219e-12,3.649707688279346e-06,-2.235508659360595e-12,-2.419651971992219e-05,-4.085545380982910e-12,-3.008912019059268e-12,1.233101682236114e-06,-2.789828271210607e-13,-8.016466720638071e-06,2.127794964530082e-12,-9.791810646039056e-14,-4.201887195312219e-13,1.326748264926616e-12,-1.620795323715701e-06,-7.040663479673701e-12,-2.462123546232446e-12,7.124729646486233e-07,5.659593818111899e-12,-7.634047594440513e-12,-9.587773835750989e-13,2.369845715420387e-12,6.473655328436402e-12,7.192407394417352e-13,3.888248381770844e-06,7.580799141553309e-12,-3.077213348748333e-05,6.138449018398182e-13,-4.891957148624818e-13,1.301488632767507e-06,1.017494703369749e-11,-1.044156374136803e-05,1.032686461890726e-12,-1.794332185552394e-09,-6.267673350018047e-07,-1.821888775590491e-12,-5.127261580698555e-08,1.858191323650427e-05,-5.049388459646141e-12,9.860142712802118e-07,1.882644608479958e-13,-3.286636923044565e-12,4.239747024477490e-13,8.844729950396789e-12,-2.361186942048178e-05,1.051356188962276e-11,-1.831171275036910e-06,4.254548464327886e-12,5.448056153241602e-12,-8.016466265447319e-06,2.215828982850329e-12,-6.162901169382998e-07,3.498841754193298e-13,1.387713296018495e-14,1.032183710561232e-13,6.579429912568604e-13,8.408245650016724e-06,-1.389127675976463e-13,7.653414652902397e-12,-3.252655718014434e-06,5.744205793174893e-13,2.141940588345653e-13,7.018729333479960e-12,1.443891199593835e-13,-1.491485460299752e-12,-3.177112500162614e-12,-3.152305589843692e-05,4.335645240011348e-12,-2.090160095159639e-06,7.504520059373343e-12,-2.474559436752278e-12,-1.044156381144262e-05,4.260736159149006e-13,-7.142187957434337e-07,-3.490159973403828e-13,4.124922923607428e-08,4.768951380587119e-12,-3.165258269791688e-12,4.890726276889666e-13,1.620626591655081e-13,-3.070993171019020e-12,-8.888405332411375e-13,8.900833311201156e-12,-1.957320534591491e-09,5.490653944329916e-12,-2.357765500176726e-09,-2.597769095572063e-12,-1.040389910970200e-12,-1.995401243272129e-12,5.723406031290502e-12,-7.740675812918631e-08,-1.147352077261521e-13,1.284475648390050e-12,-6.445008231582863e-14,9.609268893115583e-13,-2.599276886122561e-08,-1.311354849129036e-12,-1.371875941116029e-12,5.846407210760373e-07,1.411938583051173e-13,1.056407223298519e-12,-7.509144690213373e-07,-6.302851016672365e-12,-9.518984187719401e-14,1.262570960102781e-12,2.825803072448629e-12,-1.824004146386937e-12,1.216695445110104e-11,-1.192789302087662e-08,1.085475078662900e-11,1.484511348711833e-07,3.344158840094882e-11,2.624861310842660e-13,-1.793811151782465e-09,1.012016329838824e-12,4.124830448174015e-08,1.525325840303082e-12,-3.647830411591522e-08]
vect = np.array(vect, dtype=np.float32)


# Aux functions
r_eig = lambda x: np.sort(np.real(np.linalg.eigvals(x.todense())))
k = 2 # Número de clusters

def sparsity_error(A, k=2):
    # L1
    sparsity_term = np.sum(np.abs(A))

    # Clustering 
    non_zero_coeffs = A[np.abs(A) > 1e-5]  # Sacamos los ceros
    if len(non_zero_coeffs) > 0:
        # Calculamos los clusters (k) y la distancia en ellos
        centroids, _ = kmeans(non_zero_coeffs, k)
        cluster_labels, distances = vq(non_zero_coeffs, centroids)
        cluster_error = np.sum(distances**2) 
    else:
        cluster_error = 0

    loss = 1 * sparsity_term + 2 * cluster_error
    return loss

# Loss definition
def loss_fun(vect):
    # Rho 2 loss
    rho = rho_m(vect, rho_2_arrays)
    eigv = r_eig(rho)
    lrho2 = np.linalg.norm(rho_2_obj-eigv)
    # Rho 1 loss
    rho = rho_m(vect, rho_1_arrays)
    eigv = r_eig(rho)
    lrho1 = np.linalg.norm(rho_1_obj-eigv)
    # L1 + clustering loss
    l1 = sparsity_error(vect, k) 

    print(lrho2, lrho1, l1)
    return lrho2 + lrho1 + 1/2*l1

opt = scipy.optimize.minimize(loss_fun, vect, method='L-BFGS-B')
print(opt)
print(opt.x)
print(rho_2_obj)
print(sparsity_error(opt.x, k=k))
r_eig(rho_m(opt.x, rho_2_arrays))

0.16301377914072188 0.06636629488540911 3.2165838479995728
0.16301377430429514 0.0663662932122464 3.2165838479995728
0.16301377914072365 0.06636629488540935 3.2165838479995728
0.16301376464209613 0.06636628896156949 3.2165838479995728
0.16301379172297994 0.06636630002570164 3.2165838479995728
0.1630137791407221 0.06636629488540892 3.2165838479995728
0.16305613721884749 0.06638221517737093 3.217311441898346
0.16301377914072243 0.06636629488540885 3.2165838479995728
0.16301377914072004 0.06636629488540906 3.2165838479995728
0.16301377914072243 0.06636629488540854 3.2165838479995728
0.16301377914072337 0.06636629488540897 3.2165838479995728
0.1630137791563672 0.066366294890203 3.2165838479995728
0.16301377914072032 0.06636629488540857 3.2165838479995728
0.16301377908634704 0.06636629486415033 3.2165838479995728
0.16301377914072251 0.06636629488540917 3.2165838479995728
0.16301377914072324 0.06636629488540936 3.2165838479995728
0.16301377914078033 0.06636629488542253 3.2165838479995728
0.1

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.12947698e-05, 8.15400125e-05, 8.38223564e-05,
       1.99942367e-03, 3.95013237e-03, 3.97599854e-03, 8.04989026e-02,
       8.10394408e-02, 8.15324131e-02, 8.24429451e-02, 8.35738973e-02,
       8.37550551e-02, 8.46669920e-02, 8.49706094e-02, 8.52263488e-02,
       8.76611499e-02, 8.82480973e-02, 8.95068988e-02, 3.29592881e-01,
       3.32089883e-01, 3.32474387e-01, 3.33024858e-01, 3.34225991e-01,
       3.37173921e-01, 4.92135200e-01, 4.96006243e-01, 4.96016202e-01,
      

In [32]:
est = 0
app_r = lambda x: np.round(1/(x**2), 0)
for i, ii in enumerate(opt.x):
    if np.abs(ii) > 0.1:
        print(i, ii, basis.base[i])
        if np.abs(ii) > 0.2:
            print(app_r(ii))
            est += 1/np.sqrt(app_r(ii)) * basis.canonicals[i]
            print('')

#(1/np.sqrt(3), 1/np.sqrt(12))
#print(est)
est = 1/np.sqrt(12) * (-basis.canonicals[5]+basis.canonicals[25]+basis.canonicals[85]-basis.canonicals[105])+1/np.sqrt(3)*(basis.canonicals[45]+basis.canonicals[65])

print(r_eig(rho_m(est, rho_2_arrays)))


5 -0.27936976320734497 1.0 [0^ 1^ 2^ 3^ 4^ 7^ 8^ 9^ 12^ 13^]
13.0

25 0.28996316812175205 1.0 [0^ 1^ 2^ 3^ 5^ 7^ 8^ 9^ 11^ 13^]
12.0

45 0.5718530568911167 1.0 [0^ 1^ 2^ 3^ 6^ 7^ 8^ 9^ 11^ 12^]
3.0

65 0.572154480816062 1.0 [0^ 1^ 2^ 4^ 5^ 7^ 8^ 9^ 10^ 13^]
3.0

85 0.2897326805359033 1.0 [0^ 1^ 2^ 4^ 6^ 7^ 8^ 9^ 10^ 12^]
12.0

105 -0.27954194550717076 1.0 [0^ 1^ 2^ 5^ 6^ 7^ 8^ 9^ 10^ 11^]
13.0

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.5
 0.5        0.5        0.5        0.5        0

In [259]:
r_eig(rho_m(vect, rho_2_arrays))

array([6.39649654e-18, 2.27759896e-17, 2.27759896e-17, 2.97340303e-17,
       1.05847838e-08, 1.05847839e-08, 8.12469946e-02, 8.12469946e-02,
       8.12469946e-02, 8.12469946e-02, 8.12469946e-02, 8.12469946e-02,
       8.26670825e-02, 8.26670825e-02, 8.26670825e-02, 8.26670825e-02,
       8.26670825e-02, 8.26670825e-02, 3.36085923e-01, 3.36085923e-01,
       3.36085923e-01, 3.36085923e-01, 3.36085923e-01, 3.36085923e-01,
       7.48001237e-01, 7.48001237e-01, 7.51998753e-01, 7.51998753e-01])

#### Testing

In [256]:
vect = [-4.646928018967011e-01,2.770294629368069e-13,6.042253100720720e-13,-6.024516357133285e-13,-2.567187786718633e-13,1.437594192304828e-01,-2.763730452330614e-13,4.646928018967681e-01,6.783135520862164e-13,6.755585098225312e-13,-1.437594192304206e-01,2.565069368880155e-13,-6.036534951757788e-13,-6.777043000828106e-13,-6.255919748617283e-14,-2.875188384604817e-01,-6.292320178844996e-13,-5.606330583371935e-13,6.014298958703158e-13,-6.759194003610720e-13,-2.875188384604817e-01,-6.207393804663961e-14,-6.269833844031960e-13,5.590088393571927e-13,2.557458444682729e-13,-1.437594192304207e-01,6.287588334290853e-13,6.270036860946324e-13,4.000749631766127e-01,-2.383049928519640e-13,1.437594192304827e-01,-2.559215089660516e-13,5.600763473792744e-13,-5.580405389843103e-13,2.382296361695720e-13,-4.000749631765536e-01]
vect = np.array(vect)
vect

array([-4.64692802e-01,  2.77029463e-13,  6.04225310e-13, -6.02451636e-13,
       -2.56718779e-13,  1.43759419e-01, -2.76373045e-13,  4.64692802e-01,
        6.78313552e-13,  6.75558510e-13, -1.43759419e-01,  2.56506937e-13,
       -6.03653495e-13, -6.77704300e-13, -6.25591975e-14, -2.87518838e-01,
       -6.29232018e-13, -5.60633058e-13,  6.01429896e-13, -6.75919400e-13,
       -2.87518838e-01, -6.20739380e-14, -6.26983384e-13,  5.59008839e-13,
        2.55745844e-13, -1.43759419e-01,  6.28758833e-13,  6.27003686e-13,
        4.00074963e-01, -2.38304993e-13,  1.43759419e-01, -2.55921509e-13,
        5.60076347e-13, -5.58040539e-13,  2.38229636e-13, -4.00074963e-01])

In [167]:
np.sort(np.linalg.eigvals(rho_m(vect, rho_2_arrays).todense()))

array([6.39649654e-18+0.00000000e+00j, 2.27759896e-17-1.60872919e-17j,
       2.27759896e-17+1.60872919e-17j, 2.97340303e-17+0.00000000e+00j,
       1.05847838e-08+0.00000000e+00j, 1.05847839e-08+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       7.48001237e-01+0.00000000e+00j, 7.48001237e-01+0.00000000e+00j,
       7.51998753e-01+0.00000000e+00j, 7.51998753e-01+0.00000000e+00j])